In [1]:
'''VoidFinder - Hoyle & Vogeley (2002)'''



################################################################################
#
# If you have control over your python environment, voidfinder can be installed
# as a normal python package via 'python setup.py install', in which case the 
# below import of 'sys' and 'sys.path.insert(0, '/abspath/to/VoidFinder/python'
# is unnecessary.  If you aren't able to install the voidfinder package,
# you can use the sys.path.insert to add it to the list of available packages
# in your python environment.
#
# Alternately, "python setup.py develop" will 'install' some symlinks which
# point back to the current directory and you can run off the same voidfinder
# repository that you're working on as if it was installed
#
################################################################################


#import sys
#sys.path.insert(1, 'local/path/VAST/VoidFinder/vast/voidfinder/')

################################################################################
#
#   IMPORT MODULES
#
################################################################################

from vast.voidfinder import find_voids, filter_galaxies

from vast.voidfinder.multizmask import generate_mask
from vast.voidfinder.preprocessing import file_preprocess
from vast.voidfinder.table_functions import to_vector, to_array

import pickle
import numpy as np

In [2]:
################################################################################
#
#   USER INPUTS
#
################################################################################


# Number of CPUs available for analysis.
# A value of None will use one less than all available CPUs.
num_cpus = 1

#-------------------------------------------------------------------------------
survey_name = 'vollim_dr7_cbp_102709_'

# File header
# Change these directory paths to where your data is stored, and where you want 
# the output to be saved.
in_directory = '/global/homes/s/schampat/Voids/Void_analysis/Data/'
out_directory = '/global/homes/s/schampat/Voids/Voidfinder_Output/SDSS Output'


# Input file name
galaxies_filename = 'deltatable.fits'  # File format: RA, dec, redshift, comoving distance, absolute magnitude
#-------------------------------------------------------------------------------

#-------------------------------------------------------------------------------
# Survey parameters
# Note: These can be set to None, in which case VoidFinder will use the limits 
# of the galaxy catalog.
min_z = 0
max_z = 0.107

# Cosmology (uncomment and change values to change cosmology)
# Need to also uncomment relevent inputs in function calls below
Omega_M = 0.26
#h = 1

# Uncomment if you do NOT want to use comoving distances
# Need to also uncomment relevent inputs in function calls below
dist_metric = 'comoving'
#-------------------------------------------------------------------------------

#-------------------------------------------------------------------------------
# Uncomment if you do NOT want to remove galaxies with Mr > -20
# Need to also uncomment relevent input in function calls below
#mag_cut = False


# Uncomment if you do NOT want to remove isolated galaxies
# Need to also uncomment relevent input in function calls below
#rm_isolated = False
#-------------------------------------------------------------------------------

In [3]:
################################################################################
#
#   PREPROCESS DATA
#
################################################################################

galaxy_data_table, dist_limits, out1_filename, out2_filename = file_preprocess(galaxies_filename, 
                                                                               in_directory, 
                                                                               out_directory,
                                                                               mag_cut=False,
                                                                               #rm_isolated=rm_isolated,
                                                                               dist_metric=dist_metric,
                                                                               #min_z=min_z, 
                                                                               #max_z=max_z,
                                                                               Omega_M=Omega_M,
                                                                               #h=h,
                                                                               verbose=0)

print("Dist limits: ", dist_limits)

Loading galaxy data table at:  /global/homes/s/schampat/Voids/Void_analysis/Data/deltatable.fits
Galaxy data table load time:  0.008092403411865234
Calculating Rgal data table column
Finished Rgal calculation time:  0.09394598007202148
Dist limits:  [3716.344 4981.063]


In [4]:
################################################################################
#
#   GENERATE MASK
#
################################################################################

mask, mask_resolution = generate_mask(galaxy_data_table, 
                                      max_z, 
                                      dist_metric=dist_metric, 
                                      smooth_mask=True,
                                      #h=h,
                                     )


temp_outfile = open(out_directory + survey_name + 'mask.pickle', 'wb')
pickle.dump((mask, mask_resolution), temp_outfile)
temp_outfile.close()

Generating mask


In [5]:
################################################################################
#
#   FILTER GALAXIES
#
################################################################################

temp_infile = open(out_directory + survey_name + 'mask.pickle', 'rb')
mask, mask_resolution = pickle.load(temp_infile)
temp_infile.close()

wall_coords_xyz, field_coords_xyz, hole_grid_shape, coords_min = filter_galaxies(galaxy_data_table,
                                                                                 survey_name,
                                                                                 out_directory,
                                                                                 mag_cut=False,
                                                                                 dist_limits=dist_limits,
                                                                                 #mag_cut_flag=mag_cut,
                                                                                 #rm_isolated_flag=rm_isolated,
                                                                                 #hole_grid_edge_length=5.0,
                                                                                 dist_metric=dist_metric,
                                                                                 #h=h,
                                                                                 verbose=0)

del galaxy_data_table


temp_outfile = open(survey_name + "filter_galaxies_output.pickle", 'wb')
pickle.dump((wall_coords_xyz, field_coords_xyz, hole_grid_shape, coords_min), temp_outfile)
temp_outfile.close()

Filter Galaxies Start
    ra       dec     redshift    Delta        Rgal  
--------- --------- --------- ------------ ---------
2.4381757 1.4254737 1.9605263     4.355421  3716.344
2.4381757 1.4254737 1.9625721     1.663106 3718.5845
2.4381757 1.4254737 1.9646193    1.2053988  3720.824
2.4381757 1.4254737 1.9666679    1.8245617 3723.0637
2.4381757 1.4254737 1.9687178   -1.0706358 3725.3025
2.4381757 1.4254737 1.9707693    1.4216183 3727.5408
2.4381757 1.4254737 1.9728222   0.71694505 3729.7786
2.4381757 1.4254737 1.9748764  -0.36280432 3732.0154
2.4381757 1.4254737  1.976932   -0.7164737  3734.252
2.4381757 1.4254737 1.9789891   0.50402004  3736.488
      ...       ...       ...          ...       ...
 2.356985 1.4730321 2.4061904   0.14506601 4159.6436
 2.356985 1.4730321  2.408544      0.47213 4161.7686
 2.356985 1.4730321 2.4108994   0.11319674  4163.893
 2.356985 1.4730321 2.4132564   0.49564514  4166.017
 2.356985 1.4730321  2.415615  -0.45444122 4168.1406
 2.356985 1.4730321 2.41

In [6]:
print(wall_coords_xyz.dtype)

float32


In [7]:
################################################################################
#
#   FIND VOIDS
#
################################################################################


temp_infile = open(survey_name + "filter_galaxies_output.pickle", 'rb')
wall_coords_xyz, field_coords_xyz, hole_grid_shape, coords_min = pickle.load(temp_infile)
temp_infile.close()



find_voids(wall_coords_xyz,
           coords_min.astype(np.float64),
           hole_grid_shape,
           survey_name,
           min_dist = dist_limits[0],
           max_dist = dist_limits[1],
           mask=mask, 
           mask_resolution=mask_resolution,
           #save_after=50000,
           #use_start_checkpoint=True,
           #hole_grid_edge_length=5.0,
           #galaxy_map_grid_edge_length=None,
           #hole_center_iter_dist=1.0,
           maximal_spheres_filename=out1_filename,
           void_table_filename=out2_filename,
           potential_voids_filename=out_directory + survey_name + 'potential_voids_list.txt',
           num_cpus=num_cpus,
           batch_size=10000,
           verbose=0,
           print_after=5.0)

Growing holes
Processed 9118 cells of 41674 empty cells 5.15
Processed 18133 cells of 41674 empty cells 11.44
Processed 37187 cells of 41674 empty cells 18.47
WORKER EXITING GRACEFULLY 0
Found a total of 8714 potential voids.
Time to grow all holes: 20.238237857818604
Starting volume cut
Found  2292 holes to cut 0.19937849044799805
Sorting holes based on radius
Combining holes into unique voids
Starting hole combine
Removing duplicate spheres: 0.0001697540283203125
Find maximal holes: 0.0019583702087402344
Merging holes into voids: 0.0052759647369384766
Combine time: 0.011632204055786133
Number of unique voids is 112
Saving maximals to /global/homes/s/schampat/Voids/Voidfinder_Output/SDSS Outputdeltatable._comoving_maximal_noMagCut.txt


In [9]:
'''VoidFinder - Hoyle & Vogeley (2002)'''








'VoidFinder - Hoyle & Vogeley (2002)'